## Basic setup

To install the library, we can use either `PyPI` or `Conda`:

````bash
    # Note this is an alternative to the official repository.
    # It will be uploaded to the official PyPI repository.
    pip install -i https://test.pypi.org/simple/ anguilla 
````

We then import the required libraries.

In [ ]:
import numpy as np

from anguilla.fitness import benchmark
from anguilla.optimizers.mocma import MOCMA

We set up a seed for reproducibility of the results.

In [ ]:
seed = 54545
rng = np.random.default_rng(seed)

## Instantiating a benchmark function

To run the optimizer we need a function to optimize.

The library provides implementations of functions from various benchmarks used in MOO.
For this tutorial we will use the DTLZ1$: \mathbb{R}^{30} \mapsto \mathbb{R}^3$ benchmark function.

In [ ]:
n_dimensions = 30
n_objectives = 3
fn = benchmark.DTLZ1(n_dimensions, n_objectives, rng=rng)

## Instanting the optimizer

The optimizer infers the number of parents, dimensions and objectives
from the initial parent_point and parent_fitness arrays we pass its constructor 
at instantiation.

In [ ]:
n_parents = 100
parent_points = fn.random_points(n_parents)
parent_fitness = fn(parent_points)

We can omit providing a value for `n_offspring` when instantiating the optimizer.
If `n_parents = 100`, then we would've be constructing an instance of $(100+100)$-MOCMA-ES 
If we'd like to use the steady-state variant, i.e $(100+1)-MOCMA-ES
Then we need to set `n_offspring=1`.

In [ ]:
n_offspring = 100

We are ready to instantiate the MOCMA optimizer.
Note that we need to specify at least one stopping condition,
in this case we restrict the number of function evaluations by
setting `max_evaluations=50000`. Other stopping conditions are
`max_generations` and `target_indicator_value` (which requires providing a reference point).

In [ ]:
optimizer = MOCMA(parent_points,
                  parent_fitness,
                  n_offspring=n_offspring,
                  rng=rng,
                  max_evaluations=50000,
                 )

To pass a reference point, we would've done the following:

```python
our_reference_point = None
optimizer.indicator.reference = our_reference_point
```

## The `fmin` interface

Now, to optimize we can simply call the `fmin` method on the optimizer instance:

In [ ]:
result = optimizer.fmin(fn)

The optimizer returns an instance of `OptimizerResult`, which contains both the solution and the stopping conditions that were triggered.

We can access the approximated Pareto set and front by:

```python
pareto_set = result.solution.points
pareto_front = result.solution.fitness
```

We can inspect which condition(s) triggered the stop by:
    
```python
result.stopping_conditions
```

## The `ask` and `tell` interface

An alternative interface, called the ask-and-tell interface, allows for decoupled
execution between calls to the optimizer and the function.
The `fmin` implementation uses the ask-and-tell interface under the hood.

The `ask` method generates new search points (offspring) by using mutation
operators on the parent population.

The `tell` method is used to inform the optimizer about the fitness of these
new points. It then performs environmental selection to produce the new parent population.

When could we need to use this interface? Say, for example, that we want to checkpoint solutions every `5000` function evaluations:

In [ ]:
while not optimizer.stop.triggered:
    points = optimizer.ask()
    if fn.has_constraints:
        fitness, penalized_fitness = fn.evaluate_with_penalty(points)
        optimizer.tell(fitness, penalized_fitness)
    else:
        fitness = fn(points)
        optimizer.tell(fitness)
    if optimizer.evaluation_count % 5000 == 0:
        # Access current Pareto front approximation
        # and save it to a file
        np.savetxt(f"./output/fitness-{optimizer.evaluation_count}.csv",
                   optimizer.best.fitness,
                  )

Note that we treated constrained functions a bit differently in the above example.
This is because we should penalize search points that violate any constraints.
To inform the optimizer about this, you can pass the penalized fitness.

In the example above you've could also done:

```python
fitness = fn.evaluate_with_penalty(points)
optimizer.tell(*fitness)
```

In addition, some functions are noisy. Usually, these functions are evaluated multiple times 
and an average of their fitness is reported to the optimizer. This takes more our the function evaluations
budget we may define as a stopping condition. To inform the optimizer about this, we could do:

```python
    for i in range(n_repetitions):
        fitness[i] = noisy_function(point)
    optimizer.tell(np.average(fitness, axis=0), evaluation_count=n_evaluations) 
```

After the optimizer finishes the optimization run, we can get the solution by:

In [ ]:
solution = optimizer.best

## References

Pending